## elections_to_full_data

This code file departs from the election list. We implement several steps in this file. The first objective is to match the names of the elections with the names of the politicians, and also with the names of the non-politicians whose data we have collected. 

The first thing we do is load the packages, and then clean up the dataset and compute the margin of victory of all the politicians. 

In [1]:
# Load the libraries
import pandas as pd
import numpy as np
import re

from pandas_ods_reader import read_ods
import statistics

import numpy as np
import matplotlib.pyplot as plt

from matplotlib import pyplot as plt

from tqdm import tqdm

from functions import *

import fuzzywuzzy as fw
from fuzzy_match import match
from fuzzy_match import algorithims
from fuzzywuzzy import process

/home/bas/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# Import the election data
elected_people = pd.read_csv("../Data/elections/allelected.csv")

In [3]:
elected_people['naam'] = elected_people['voornaam'] + ' ' + elected_people['achternaam']
elected_people['verkiezingdatum'] = (elected_people['dag'].astype(str) + 
                                     '-' + 
                                     elected_people['maand'].astype(str) +
                                     '-' +
                                     elected_people['jaar'].astype(str)
                                    )
elected_people['verkiezingdatum'] = elected_people['verkiezingdatum'].apply(lambda x: pd.Timestamp(x))

In [4]:
election_results_details = pd.read_csv("../Data/elections/election_results_details.csv").iloc[:,1:]
election_results_details['Verkiezingdatum'] = (election_results_details['Verkiezingdatum'].
                                               apply(lambda x: pd.Timestamp(x))
                                              )

def get_zetels(df):
    a = pd.to_numeric(df['Aantal zetels'], errors='coerce')
    b = np.mean(a)
    return (b)

aantal_zetels = (election_results_details.groupby(['District', 'Verkiezingdatum']).
                 apply(get_zetels).reset_index().rename(columns={0:'Aantal zetels'})
                )

In [5]:
pd.merge(elected_people, aantal_zetels, 
         left_on=['districtsnaam', 'verkiezingdatum'],
         right_on=['District', 'Verkiezingdatum']).drop(columns=['District', 'Verkiezingdatum'])

,achternaam,voornaam,tussenvoegsel,jaar,maand,dag,type verkiezing,districtsnaam,aantal stemmen,omvang_electoraat,zetels,drempel,naam,verkiezingdatum,Aantal zetels
0,Aalberse,P.J.M.,NaN,1903,2,18,tussentijds,Almelo,3821,7865,1.0,2953.0,P.J.M. Aalberse,1903-02-18,1.0
1,Aalberse,P.J.M.,NaN,1905,6,16,algemeen,Almelo,5217,9324,1.0,3922.0,P.J.M. Aalberse,1905-06-16,1.0
2,Aalberse,P.J.M.,NaN,1909,6,11,algemeen,Almelo,4744,9087,1.0,3498.0,P.J.M. Aalberse,1909-11-06,1.0
3,Aalberse,P.J.M.,NaN,1913,6,17,algemeen,Almelo,5338,10107,1.0,3734.0,P.J.M. Aalberse,1913-06-17,1.0
4,Akerlaken,D. van,NaN,1850,8,27,algemeen,Hoorn,1319,3430,2.0,1122.0,D. van Akerlaken,1850-08-27,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2423,Zuylen van Nijevelt,J.P.J.A. graaf van,NaN,1871,6,13,periodiek,Arnhem,1009,2863,1.0,989.0,J.P.J.A. graaf van Zuylen van Nijevelt,1871-06-13,1.0
2424,Zwaag,G.L. van der,NaN,1897,6,25,herstemming,Schoterland,1941,5584,1.0,1796.0,G.L. van der Zwaag,1897-06-25,1.0
2425,Zwaag,G.L. van der,NaN,1901,6,27,herstemming,Weststellingwerf,1984,5228,1.0,1765.0,G.L. van der Zwaag,1901-06-27,1.0
2426,Zwaag,G.L. van der,NaN,1901,6,27,herstemming,Schoterland,2168,5184,1.0,1809.0,G.L. van der Zwaag,1901-06-27,1.0


In [6]:
all_candidates = pd.read_csv("../Data/elections/election_results_details.csv").iloc[:,1:]
all_candidates['Aantal zetels'] = all_candidates['Aantal zetels'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
all_candidates['Verkiezingdatum'] = (all_candidates['Verkiezingdatum'].
                                     apply(lambda x: pd.Timestamp(x))
                                    )
all_candidates['Aantal stemmen'] = (all_candidates['Aantal stemmen'].
                                    apply(lambda x: pd.to_numeric(x, errors='coerce'))
                                   )

aantal_stemmen = (all_candidates.groupby(['District','Verkiezingdatum']).
 apply(lambda x: sum(x['Aantal stemmen'])).
 reset_index().
 rename(columns={0:'totaal aantal stemmen'})
)

all_candidates = pd.merge(all_candidates, aantal_stemmen,
        left_on=['District', 'Verkiezingdatum'],
        right_on=['District', 'Verkiezingdatum'])

all_candidates = (all_candidates.groupby(['District', 'Verkiezingdatum']).
 apply(lambda x: x.sort_values(['Aantal stemmen'], 
                               ascending = False))
).reset_index(drop=True)

all_candidates['hoeveelste_in_verkiezing'] = (all_candidates.groupby(['District', 'Verkiezingdatum']).
                                              cumcount() + 1)

all_candidates['gewonnen'] = np.where(all_candidates['hoeveelste_in_verkiezing'] <= all_candidates['Aantal zetels'], 1, 0)
all_candidates['marginal_winner'] = np.where(all_candidates['Aantal zetels'] - all_candidates['hoeveelste_in_verkiezing'] == 0, 1, 0)
all_candidates['marginal_loser'] = np.where(all_candidates['Aantal zetels'] - all_candidates['hoeveelste_in_verkiezing'] == -1, 1, 0)

In [7]:
all_candidates = get_margin(all_candidates)

100%|██████████| 8238/8238 [01:22<00:00, 100.21it/s]


In [8]:


# hoe vaak gewonnen in totaal (algemeen)

# hoeveelste keer proberen deze verkiezing (algemeen)

# hoeveelste keer gewonnen deze verkiezing (algemeen)

# volgende verkiezing gewonnen ja/nee

# 2, 3, 4e volgende verkiezing gewonnen ja/nee

def get_elec_stats(dataframe):
    
    out = pd.DataFrame()
    
    for i in tqdm(range(len(dataframe))):
        
        naam = dataframe.iloc[i]['Naam']
        datum = dataframe.iloc[i]['Verkiezingdatum']
        
        # proberen
        hoeveelste_keer_prob = dataframe[(dataframe['Naam'] == naam) & (dataframe['Verkiezingdatum'] <= datum)].shape[0]
        hoeveelste_keer_prob_alg_per = dataframe[(dataframe['Naam'] == naam) & (dataframe['Verkiezingdatum'] <= datum) & ( dataframe['Type'] == 'algemeen' | dataframe['Type'] == 'periodiek')].shape[0]
        hoeveelste_keer_alg = dataframe[(dataframe['Naam'] == naam) & (dataframe['Verkiezingdatum'] <= datum) & ( dataframe['Type'] == 'algemeen')].shape[0]
        
        # gewonnen
        hoeveelste_keer_gewonnen = dataframe[(dataframe['Naam'] == naam) & (dataframe['gewonnen'] == 1) & (dataframe['Verkiezingdatum'] <= datum)].shape[0]
        hoeveelste_keer_gewonnen_alg = dataframe[(dataframe['Naam'] == naam) & (dataframe['gewonnen'] == 1) & (dataframe['Verkiezingdatum'] <= datum) & (dataframe['Type'] == 'algemeen')].shape[0]
        hoevaak_gewonnen_totaal = 
        
        # volgende verkiezingen (met np.where(None, if niet meedoen))
        verk_2_gewonnen
        verk_3_gewonnen
        verk_4_gewonnen
        verk_5_gewonnen
        verk_6_gewonnen
        

,Naam,Aanbevolen door,Aantal stemmen,Procentueel,District,Verkiezingdatum,Type,Omvang electoraat,Opkomst,Aantal stembriefjes,Aantal stemmen geldig,Aantal stemmen blanco,Aantal zetels,Kiesdrempel,totaal aantal stemmen,hoeveelste_in_verkiezing,gewonnen,marginal_winner,marginal_loser,margin
0,S.A. de Moraaz,NaN,503.0,52.84%,Alkmaar,1848-11-30,algemeen,1107,956,956,952,4,1.0,476,941.0,1,1,1,0,0.069075
1,G. van Leeuwen,NaN,438.0,46.01%,Alkmaar,1848-11-30,algemeen,1107,956,956,952,4,1.0,476,941.0,2,0,0,1,-0.069075
2,mr. H.J. Smit,NaN,1566.0,79.86%,Alkmaar,1850-08-27,algemeen,2833,1977,1979,3922,20,2.0,981,3781.0,1,1,0,0,0.233007
3,S.A. de Moraaz,NaN,1275.0,65.02%,Alkmaar,1850-08-27,algemeen,2833,1977,1979,3922,20,2.0,981,3781.0,2,1,1,0,0.156043
4,jhr.mr. C. van Foreest,NaN,685.0,34.93%,Alkmaar,1850-08-27,algemeen,2833,1977,1979,3922,20,2.0,981,3781.0,3,0,0,1,-0.156043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8233,B. Luteraan,SDP,32.0,0.41%,Zwolle,1913-06-17,algemeen,8889,-,7897,7786,111,1.0,3893,7786.0,4,0,0,0,-0.494606
8234,F.M. Knobel,VL(Lib/VD/SDAP),4249.0,50.15%,Zwolle,1913-06-25,herstemming,8889,-,8516,8472,44,1.0,4236,8472.0,1,1,1,0,0.003069
8235,A. baron van Dedem,CHU(Ka/AR),4223.0,49.85%,Zwolle,1913-06-25,herstemming,8889,-,8516,8472,44,1.0,4236,8472.0,2,0,0,1,-0.003069
8236,F.M. Knobel,VL,3236.0,86.22%,Zwolle,1917-06-15,algemeen,9645,-,3967,3753,214,1.0,1877,3753.0,1,1,1,0,0.724487


In [13]:
all_candidates[(all_candidates['Naam'] == 'S.A. de Moraaz') & (all_candidates['Verkiezingdatum'] > '1850-11-30')].shape[0]

2